In [13]:
import pandas as pd
import os


pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
print("✅ Entorno configurado.")

✅ Entorno configurado.


In [14]:
folder_path = '../data/raw/'

try:
    df_orders = pd.read_csv(os.path.join(folder_path, 'olist_orders_dataset.csv'))
    df_items = pd.read_csv(os.path.join(folder_path, 'olist_order_items_dataset.csv'))
    df_products = pd.read_csv(os.path.join(folder_path, 'olist_products_dataset.csv'))
    df_customers = pd.read_csv(os.path.join(folder_path, 'olist_customers_dataset.csv'))
    
    print("✅ Tablas cargadas correctamente.")
except FileNotFoundError as e:
    print(f"❌ Error crítico: Archivo no encontrado. {e}")

✅ Tablas cargadas correctamente.


In [15]:
df_total = df_items.merge(df_orders, on='order_id', how='inner')
df_total = df_total.merge(df_products, on='product_id', how='inner')
df_total = df_total.merge(df_customers, on='customer_id', how='inner')

print(f"✅ Unión completada. Dimensiones crudas: {df_total.shape}")

✅ Unión completada. Dimensiones crudas: (112650, 26)


In [16]:
print("--- INICIANDO LIMPIEZA DE DATOS ---")

# A) Fechas a tipo Datetime
cols_date = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in cols_date:
    df_total[col] = pd.to_datetime(df_total[col], errors='coerce')

# B) Imputación de nulos en variables categóricas
df_total['product_category_name'] = df_total['product_category_name'].fillna('Otros')

# C) Creación de KPIs (Feature Engineering)
# Días reales de entrega
df_total['delivery_days'] = (df_total['order_delivered_customer_date'] - df_total['order_purchase_timestamp']).dt.days
# Booleano de retraso en la entrega
df_total['is_late'] = df_total['order_delivered_customer_date'] > df_total['order_estimated_delivery_date']
# Formato Mes-Año en texto puro (Evita que Power BI formatee fechas largas)
df_total['month_year'] = df_total['order_purchase_timestamp'].dt.strftime('%Y-%m')

# D) Filtrado de Calidad (Outliers)
df_clean = df_total[
    (df_total['delivery_days'] > 0) & 
    (df_total['delivery_days'] < 100) & 
    (df_total['product_weight_g'].notnull())
].copy()

int_columns = [
    'delivery_days', 
    'product_photos_qty', 
    'product_weight_g', 
    'product_length_cm', 
    'product_height_cm', 
    'product_width_cm', 
    'product_name_lenght', 
    'product_description_lenght'
]

for col in int_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('Int64')

ruta_guardado = '../data/processed/Olist_Business_Data_Clean.csv'
df_clean.to_csv(ruta_guardado, index=False)


print("\n--- VERIFICACIÓN DE SEGURIDAD ---")
df_check = pd.read_csv(ruta_guardado)

print(f"Archivo guardado en: {ruta_guardado}")
print(f"Filas procesadas con éxito: {len(df_check)}")
print(f"Valor MÁXIMO de días de entrega: {df_check['delivery_days'].max()}")
print(f"Ejemplo de datos corregidos (sin .0): \n{df_check['delivery_days'].head(3).tolist()}")

if df_check['delivery_days'].max() < 100:
    print("✅ PRUEBA SUPERADA: Archivo listo para importar a Power BI.")
else:
    print("❌ ALERTA: Hay un fallo en la validación de outliers.")

--- INICIANDO LIMPIEZA DE DATOS ---

--- VERIFICACIÓN DE SEGURIDAD ---
Archivo guardado en: ../data/processed/Olist_Business_Data_Clean.csv
Filas procesadas con éxito: 110091
Valor MÁXIMO de días de entrega: 99
Ejemplo de datos corregidos (sin .0): 
[7, 16, 7]
✅ PRUEBA SUPERADA: Archivo listo para importar a Power BI.
